# Clean dataset before injecting into pipeline
- In this notebook, we analyze each individual column to determine the most useful features to include in the training of our final model

In [251]:
# import dependencies

import numpy as np
import pandas as pd

import matplotlib as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_columns = 100

In [252]:
# columns that i will be dropping
drops = []

## Load in data:

In [253]:
features = pd.read_csv('data/x_train.csv')
labels = pd.read_csv('data/y_train.csv')

# merge the target col to the features df
features = features.merge(labels, on='id')

print(f'Total wells in original dataset: {features.shape[0]}\nNumber of columns: {features.shape[1]}')

Total wells in original dataset: 59400
Number of columns: 41


### Dataset:
- Almost 60,000 total wells to analyze
- 39 features (not including id or status_group)

# Null values:
- Null values are handled in model_pipeline.ipynb using sklearn's SimpleImputer and MissingIndicator
- Pseudo-null values (like fill values of 0) are set to "nan" in the "Individual column analysis" section of this notebook and are also accounted for in model_pipeline.ipynb.

In [254]:
features.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

- Most columns don't have null values
- Even the columns with nulls generally don't have many

# Add "age" column:

- Some values in the dataset may not be null, but instead filled with the placeholder of "0".

In [255]:
features.date_recorded.dtype

dtype('O')

In [256]:
# check date_recorded for missing values
features[features['date_recorded'] == '0'].shape

(0, 41)

In [257]:
features.construction_year.dtype

dtype('int64')

In [258]:
# check construction_year for missing values
features[features['construction_year'] == 0].shape

(20709, 41)

- Over 20,000 wells missing construction_year (1/3 of the dataset)
- If I want to use the age of the well as a feature, I need to fill these values with mean or median

In [259]:
features[['construction_year']].describe().T

,count,mean,std,min,25%,50%,75%,max
construction_year,59400.0,1300.652475,951.620547,0.0,0.0,1986.0,2004.0,2013.0


- Mean (year 1300) is unrealistic. Median (1986) is a much better fill value.

In [260]:
# mask for missing construction_year
mask = features['construction_year'].astype(int) == 0

# set construction_year to median (1986) if missing
features.loc[mask, 'construction_year'] = features['construction_year'].median()

# calculate age in years
features['age_years'] = features['date_recorded'].str[:4].astype(int) - features['construction_year']

In [261]:
# drop the two date columns, as age_years should capture this information
drops.append('date_recorded')
drops.append('construction_year')

# Individual column analysis:

## total static head:
- Change in elevatin between water source and discharge (where water is released). The distance that the water must travel to get from source to output.

In [262]:
features.amount_tsh.to_frame().describe().T

,count,mean,std,min,25%,50%,75%,max
amount_tsh,59400.0,317.650385,2997.574558,0.0,0.0,0.0,20.0,350000.0


- Lots of TSH of 0. Even the median (50%) value is 0. This could be due to missing values or different pump types.
- Standard deviation is massive (almost 10x the mean).

## funder:

In [263]:
features.funder.nunique()

1897

- almost 1900 unique funders
- This may be too many columns to encode

In [264]:
# get counts of num wells funded for each funder
funders = features.funder.value_counts()

# funders that have funded at least 100 wells
top_funders = funders[funders >= 100].index.tolist()
vals = funders[funders >= 100].values.tolist()

In [265]:
num_funders = len(top_funders)
num_funders

91

- 91 / 1897 funders have built more than 100 wells.

In [266]:
print(f'''The top {num_funders / features.funder.nunique() * 100 :.1f}% of funders make up \
{sum(vals) / features.shape[0] * 100 :.1f}% of the data''')

The top 4.8% of funders make up 73.0% of the data


- Taking funders with 100+ wells funded in dataset.
- This will result in less One-hot encoded columns and make training process smoother.

In [267]:
# replace other funders with "other"
features['funder'] = features['funder'].where(features['funder'].isin(top_funders), 'other')

## gps_height:
- Indicates the altitude of the well, presumably compared to sea level.

In [268]:
# cast this col from int to float
features.gps_height = features.gps_height.astype(float)

In [269]:
features[['gps_height']].describe().T

,count,mean,std,min,25%,50%,75%,max
gps_height,59400.0,668.297239,693.11635,-90.0,0.0,369.0,1319.25,2770.0


- Minimum is -90, indicating a well located below sea level.
- Max is 2770, indicating a well in an elevated region.

## installer:

In [270]:
features.installer.nunique()

2145

- Over 2100 unique installers
- Once again, too many columns to encode

In [271]:
# get counts of num wells funded for each funder
installers = features.installer.value_counts()

# installers that have installed at least 100 wells
top_installers = installers[installers >= 100].index.tolist()
vals = installers[installers >= 100].values.tolist()

In [272]:
num_installers = len(top_installers)
num_installers

84

- 84 / 2145 installers have built more than 100 wells

In [273]:
print(f'''The top {num_installers / features.installer.nunique() * 100 :.1f}% of installers make up \
{sum(vals) / features.shape[0] * 100 :.1f}% of the data''')

The top 3.9% of installers make up 71.0% of the data


- Taking installers with 100+ wells installed in dataset

In [274]:
# replace other installers with "other"
features['installer'] = features['installer'].where(features['installer'].isin(top_installers), 'other')

## longitude and latitude:

- Tanzania's longitude spans from roughly 29 to 41 degrees.
- Tanzania's latitude spans from roughly -12 to -1 degrees.
- All vals should fall in between these ranges.

In [275]:
features[['longitude']].describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,59400.0,34.077427,6.567432,0.0,33.090347,34.908743,37.178387,40.345193


- Seems like several of the longitude vals are below 29 degrees. None are over 41 degrees.

In [276]:
features[features['longitude'] < 29]['longitude'].to_frame().T

,21,53,168,177,253,256,285,301,306,321,323,326,346,370,433,659,678,697,720,733,753,755,798,839,911,939,960,965,971,992,1054,1079,1122,1168,1191,1208,1217,1240,1250,1252,1303,1333,1334,1424,1449,1454,1463,1502,1525,1549,...,57748,57789,57794,57807,57812,57823,57827,57861,57918,58005,58039,58045,58046,58071,58077,58102,58257,58397,58413,58415,58416,58435,58446,58463,58517,58545,58550,58555,58607,58678,58735,58771,58837,58859,58969,58997,59017,59033,59111,59120,59135,59158,59162,59166,59184,59189,59208,59295,59324,59374
longitude,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 1812 rows where longitude vals are 0 (invalid).

In [277]:
# set the above indices to nan
indices = features[features['longitude'] < 29].index
features.loc[indices, 'longitude'] = np.nan

In [278]:
# show that all '<29' rows are gone
features[features['longitude'] < 29]['longitude'].to_frame().T

""
longitude


In [279]:
features[['latitude']].describe().T

,count,mean,std,min,25%,50%,75%,max
latitude,59400.0,-5.706033,2.946019,-11.64944,-8.540621,-5.021597,-3.326156,-2.000000e-08


- Several of the latitude vals are greater than -1 degrees. None are below -12 degrees

In [280]:
# using -0.9 instead of -1 as there were 7 valid values in this range
features[features['latitude'] > -0.9]['latitude'].to_frame().T

,21,53,168,177,253,256,285,301,306,321,323,326,346,370,433,659,678,697,720,733,753,755,798,839,911,939,960,965,971,992,1054,1079,1122,1168,1191,1208,1217,1240,1250,1252,1303,1333,1334,1424,1449,1454,1463,1502,1525,1549,...,57748,57789,57794,57807,57812,57823,57827,57861,57918,58005,58039,58045,58046,58071,58077,58102,58257,58397,58413,58415,58416,58435,58446,58463,58517,58545,58550,58555,58607,58678,58735,58771,58837,58859,58969,58997,59017,59033,59111,59120,59135,59158,59162,59166,59184,59189,59208,59295,59324,59374
latitude,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,...,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08,-2.000000e-08


- Like longitude, 1812 rows with invalid latitude

In [281]:
# set the above indices to nan
indices = features[features['latitude'] > -0.9].index
features.loc[indices, 'latitude'] = np.nan

In [282]:
# show that all '>-0.9' rows are gone
features[features['latitude'] > -0.9]['latitude'].to_frame().T

""
latitude


## wpt_name:

In [283]:
features.wpt_name.nunique()

37400

- This column is for well name. Shouldn't have impact on pump functionality, so we're going to drop.

In [284]:
drops.append('wpt_name')

## num_private:

In [285]:
features['num_private'].describe()

count    59400.000000
mean         0.474141
std         12.236230
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       1776.000000
Name: num_private, dtype: float64

In [286]:
features.num_private.nunique()

65

In [287]:
features['num_private'].value_counts(normalize=True).head(3)

0    0.987256
6    0.001364
1    0.001229
Name: num_private, dtype: float64

- Vast majority of vals are 0

In [288]:
# get counts of rows for each num_private value
num_privates = features.num_private.value_counts()

# at least 10 rows
top_num_privates = num_privates[num_privates >= 10].index.tolist()
vals = num_privates[num_privates >= 10].values.tolist()

len(top_num_privates)

24

- 24 num_private values have at least 10 rows in dataset. May revisit this column later

In [289]:
# subset df where num_private values have at least 10 rows
at_least_10 = features[features['num_private'].isin(top_num_privates)]

In [290]:
at_least_10.groupby(['num_private'])['status_group'].value_counts(normalize=True).to_frame().T

num_private         0                                                 1    \
status_group functional non functional functional needs repair functional   
status_group    0.54223       0.384973                0.072796   0.547945   

num_private                                                2                   \
status_group non functional functional needs repair functional non functional   
status_group       0.369863                0.082192   0.608696       0.391304   

num_private         3                                                     4    \
status_group functional non functional functional needs repair non functional   
status_group   0.444444       0.444444                0.111111           0.55   

num_private                    5                         6                   \
status_group functional functional non functional functional non functional   
status_group       0.45   0.673913       0.326087   0.679012       0.246914   

num_private                                 7                   \
status_group functional needs repair functional non functional   
status_group                0.074074   0.576923       0.384615   

num_private                                 8                            \
status_group functional needs repair functional functional needs repair   
status_group                0.038462   0.673913                0.173913   

num_private                            11                    15   \
status_group non functional non functional functional functional   
status_group       0.152174       0.545455   0.454545   0.771429   

num_private                                                17   \
status_group functional needs repair non functional functional   
status_group                0.114286       0.114286   0.588235   

num_private                                                    20              \
status_group functional needs repair non functional non functional functional   
status_group                0.235294       0.176471       0.642857   0.285714   

num_private                                 25                  \
status_group functional needs repair functional non functional   
status_group                0.071429        0.5       0.416667   

num_private                                 32                            34   \
status_group functional needs repair functional non functional non functional   
status_group                0.083333      0.975          0.025            0.7   

num_private                        39                    41                  \
status_group functional non functional functional functional non functional   
status_group        0.3       0.733333   0.266667        0.7            0.3   

num_private         45                                                47   \
status_group functional non functional functional needs repair functional   
status_group   0.722222           0.25                0.027778   0.571429   

num_private                                                65                  \
status_group non functional functional needs repair functional non functional   
status_group       0.380952                0.047619        0.5            0.5   

num_private             80                    93                           \
status_group non functional functional functional functional needs repair   
status_group       0.933333   0.066667       0.75                0.214286   

num_private                        102                 
status_group non functional functional non functional  
status_group       0.035714        0.7            0.3

## basin and subvillage:

In [291]:
# unique basins
features.basin.nunique(), features.basin.unique()

(9,
 array(['Lake Nyasa', 'Lake Victoria', 'Pangani',
        'Ruvuma / Southern Coast', 'Internal', 'Lake Tanganyika',
        'Wami / Ruvu', 'Rufiji', 'Lake Rukwa'], dtype=object))

In [292]:
# unique subvillages
features.subvillage.nunique()

19287

- Going to drop subvillage. Region code, latitude, longitude, and basin should capture enough locational information.

In [293]:
drops.append('subvillage')

## region and region_code:

In [294]:
# unique regions
features.region.nunique()

21

In [295]:
# unique region_codes
features.region_code.nunique()

27

In [296]:
features.groupby(['region', 'region_code']).groups.keys()

dict_keys([('Arusha', 2), ('Arusha', 24), ('Dar es Salaam', 7), ('Dodoma', 1), ('Iringa', 11), ('Kagera', 18), ('Kigoma', 16), ('Kilimanjaro', 3), ('Lindi', 8), ('Lindi', 18), ('Lindi', 80), ('Manyara', 21), ('Mara', 20), ('Mbeya', 12), ('Morogoro', 5), ('Mtwara', 9), ('Mtwara', 90), ('Mtwara', 99), ('Mwanza', 17), ('Mwanza', 19), ('Pwani', 6), ('Pwani', 40), ('Pwani', 60), ('Rukwa', 15), ('Ruvuma', 10), ('Shinyanga', 11), ('Shinyanga', 14), ('Shinyanga', 17), ('Singida', 13), ('Tabora', 14), ('Tanga', 4), ('Tanga', 5)])

- 7 regions have multiple codes.
- Dropping "region" as it is redundant and less specific than region_code.

In [297]:
# drop 'region'
drops.append('region')

In [298]:
# check data type of region_code
features.region_code.dtypes

dtype('int64')

- Converting region_code to 'object' to ensure it's treated as a category.

In [299]:
# convert from int to str (Object)
features['region_code'] = features['region_code'].astype('O')
features.region_code.dtypes

dtype('O')

## district_code

In [300]:
# unique district_codes
features.district_code.nunique(), features.district_code.unique()

(20,
 array([ 5,  2,  4, 63,  1,  8,  3,  6, 43,  7, 23, 33, 53, 62, 60, 30, 13,
         0, 80, 67], dtype=int64))

- Like region_code, need to convert to categorical column.

In [301]:
# convert from int to str (Object)
features['district_code'] = features['district_code'].astype('O')
features.district_code.dtypes

dtype('O')

## lga and ward
- Local Government Area: an administrative division within the country
- Ward: a smaller division of an LGA

In [302]:
# unique LGA
features.lga.nunique()

125

In [303]:
# unique ward
features.ward.nunique()

2092

- Dropping 'ward' as LGA should provide enough information.

In [304]:
# drop "ward"
drops.append('ward')

## population

In [305]:
features.population.describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
population,59400.0,179.909983,471.482176,0.0,0.0,25.0,215.0,30500.0


- Min and 25th percentile both show populations of 0.

In [306]:
# populations of 0
features[features.population == 0].shape

(21381, 42)

- over 21,000 rows with a population of 0. Could be a missing value.
- could also just be wells located in remote areas with populations of 0.

In [307]:
# populations of 1
features[features.population == 1].shape

(7025, 42)

In [308]:
# populations of less than 10
features[features.population < 10].shape

(28527, 42)

- 7,000 rows with a population of 1. This may indicate that the population values are accurate.
- Almost half of our dataset has a very small population (<10). These population values very well could be accurate. 
- Keeping column unmodified.

## public_meeting
- Boolean column indicate whether a meeting was held regarding the well

In [309]:
features.public_meeting.value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

## recorded_by
- The body responsible for recording that row's data.

In [310]:
# unique recorded_by
features.recorded_by.nunique()

1

In [311]:
features.recorded_by.value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

- All well data was recorded by GeoData Consultants Ltd.
- Dropping the column.

In [312]:
# drop recorded_by
drops.append('recorded_by')

## scheme_management, management , and management_group

In [313]:
features['scheme_management'].nunique(), features['scheme_management'].unique()

(12,
 array(['VWC', 'Other', nan, 'Private operator', 'WUG', 'Water Board',
        'WUA', 'Water authority', 'Company', 'Parastatal', 'Trust', 'SWC',
        'None'], dtype=object))

In [314]:
features['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [315]:
features['management'].nunique(), features['management'].unique()

(12,
 array(['vwc', 'wug', 'other', 'private operator', 'water board', 'wua',
        'company', 'water authority', 'parastatal', 'unknown',
        'other - school', 'trust'], dtype=object))

In [316]:
features['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [317]:
features['management_group'].nunique(), features['management_group'].unique()

(5,
 array(['user-group', 'other', 'commercial', 'parastatal', 'unknown'],
       dtype=object))

In [318]:
features['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

- Scheme_management and management seem to capture the same data. Using 'management' as it has less unknown values.
- Management_group seems to also capture the same data, at a less granular level.
- Dropping scheme_management and management_group.

In [319]:
# drop scheme_management and management_group
drops.append('scheme_management')
drops.append('management_group')

## scheme_name
- The name of the scheme/program in which the well operates under.
- This is the column with the most nulls by far (28,000).

In [320]:
# unique scheme_name
features['scheme_name'].nunique()

2696

In [321]:
# get counts of scheme_names
schemes = features.scheme_name.value_counts()
schemes, schemes.sum()

(K                         682
 None                      644
 Borehole                  546
 Chalinze wate             405
 M                         400
                          ... 
 Lerang'wa waterbsupply      1
 Mtunguruzi                  1
 Heka water supply           1
 Samunge water scheme        1
 Ntang'whale                 1
 Name: scheme_name, Length: 2696, dtype: int64,
 31234)

- Only 31,000/59,000 rows have scheme_name.
- Could use a SimpleImpute strategy like "most_frequent". However, this could be innaccurate as there isn't a clear most frequent. Top value "K" barely edges out "None", and than at least 3 others are close behind.

In [322]:
# schemes with 20 or more rows in data
top_schemes = schemes[schemes >= 20].index
vals = schemes[schemes >= 20].values

num_schemes = len(top_schemes)
num_schemes

353

- 353 scheme_names with 20 or more rows.

In [323]:
print(f'''The top {num_schemes / features.scheme_name.nunique() * 100 :.1f}% of schemes make up \
{sum(vals) / 31234 * 100 :.1f}% of the data. Keep in mind, the data here is only 31,000 rows.''')

The top 13.1% of schemes make up 61.1% of the data. Keep in mind, the data here is only 31,000 rows.


- Scheme_name is more dispersed than top-heavy cols like funder and installer. This makes it harder to take the "top slice" of the column.
- Keeping this column and all of it's unique vals, as it may be very important.

## permit
- Permit is a boolean column that holds whether the builder of the well had a permit

In [324]:
# unique permits
features['permit'].nunique(), features['permit'].unique()

(2, array([False, True, nan], dtype=object))

In [325]:
features['permit'].value_counts(normalize=True)

True     0.68955
False    0.31045
Name: permit, dtype: float64

## extraction_type, extraction_type_group, extraction_type_class
- 3 columns describe the type of pump used.

In [326]:
# unique extraction_type
features['extraction_type'].nunique(), features['extraction_type'].unique()

(18,
 array(['gravity', 'submersible', 'swn 80', 'nira/tanira', 'india mark ii',
        'other', 'ksb', 'mono', 'windmill', 'afridev', 'other - rope pump',
        'india mark iii', 'other - swn 81', 'other - play pump', 'cemo',
        'climax', 'walimi', 'other - mkulima/shinyanga'], dtype=object))

In [327]:
# unique extraction_type_class
features['extraction_type_class'].nunique(), features['extraction_type_class'].unique()

(7,
 array(['gravity', 'submersible', 'handpump', 'other', 'motorpump',
        'wind-powered', 'rope pump'], dtype=object))

In [328]:
# unique extraction_type_group
features['extraction_type_group'].nunique(), features['extraction_type_group'].unique()

(13,
 array(['gravity', 'submersible', 'swn 80', 'nira/tanira', 'india mark ii',
        'other', 'mono', 'wind-powered', 'afridev', 'rope pump',
        'india mark iii', 'other handpump', 'other motorpump'],
       dtype=object))

In [329]:
features.groupby(['extraction_type_class', 'extraction_type']).groups.keys()

dict_keys([('gravity', 'gravity'), ('handpump', 'afridev'), ('handpump', 'india mark ii'), ('handpump', 'india mark iii'), ('handpump', 'nira/tanira'), ('handpump', 'other - mkulima/shinyanga'), ('handpump', 'other - play pump'), ('handpump', 'other - swn 81'), ('handpump', 'swn 80'), ('handpump', 'walimi'), ('motorpump', 'cemo'), ('motorpump', 'climax'), ('motorpump', 'mono'), ('other', 'other'), ('rope pump', 'other - rope pump'), ('submersible', 'ksb'), ('submersible', 'submersible'), ('wind-powered', 'windmill')])

- Extraction_type seems to hold the most detail with 18 unique values.
- Extraction_type_group holds similar data to extraction_type but less specific.
- Keeping extraction_type_class, as it holds broader well type data and may have meaning.
- Dropping extraction_type_group.

In [330]:
# drop extraction_type_group
drops.append('extraction_type_group')

## payment, payment_type

In [331]:
# unique payments
features['payment'].nunique(), features['payment'].unique()

(7,
 array(['pay annually', 'never pay', 'pay per bucket', 'unknown',
        'pay when scheme fails', 'other', 'pay monthly'], dtype=object))

In [332]:
features['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [333]:
# unique payment_types
features['payment_type'].nunique(), features['payment_type'].unique()

(7,
 array(['annually', 'never pay', 'per bucket', 'unknown', 'on failure',
        'other', 'monthly'], dtype=object))

In [334]:
features['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

- True duplicate cols with identical value counts.
- Dropping "payment_type".

In [335]:
# drop payment_type
drops.append('payment_type')

## water_quality, quality_group

In [336]:
# unique water_quality
features['water_quality'].nunique(), features['water_quality'].unique()

(8,
 array(['soft', 'salty', 'milky', 'unknown', 'fluoride', 'coloured',
        'salty abandoned', 'fluoride abandoned'], dtype=object))

In [337]:
features['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [338]:
# unique quality_group
features['quality_group'].nunique(), features['quality_group'].unique()

(6,
 array(['good', 'salty', 'milky', 'unknown', 'fluoride', 'colored'],
       dtype=object))

In [339]:
features['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

- Water_quality seems to be more granular with two extra unique values.
- Dropping quality_group.

In [340]:
# drop quality_group
drops.append('quality_group')

## quantity, quantity_group

In [341]:
# unique quantity
features['quantity'].nunique(), features['quantity'].unique()

(5,
 array(['enough', 'insufficient', 'dry', 'seasonal', 'unknown'],
       dtype=object))

In [342]:
features['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [343]:
# unique quantity_group
features['quantity_group'].nunique(), features['quantity_group'].unique()

(5,
 array(['enough', 'insufficient', 'dry', 'seasonal', 'unknown'],
       dtype=object))

In [344]:
features['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

- Duplicate columns.
- Dropping quantity_group

In [345]:
# drop quantity_group
drops.append('quantity_group')

## source, source_type, source_class	

In [346]:
# unique source
features['source'].nunique(), features['source'].unique()

(10,
 array(['spring', 'rainwater harvesting', 'dam', 'machine dbh', 'other',
        'shallow well', 'river', 'hand dtw', 'lake', 'unknown'],
       dtype=object))

In [347]:
features['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [348]:
# unique source_type
features['source_type'].nunique(), features['source_type'].unique()

(7,
 array(['spring', 'rainwater harvesting', 'dam', 'borehole', 'other',
        'shallow well', 'river/lake'], dtype=object))

In [349]:
features['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [350]:
# unique source_class
features['source_class'].nunique(), features['source_class'].unique()

(3, array(['groundwater', 'surface', 'unknown'], dtype=object))

In [351]:
features['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

- Keeping source_class as it distinguishes above and below ground water sources.
- Dropping source_type. "source" is more specific.

In [352]:
# drop source_type
drops.append('source_type')

## waterpoint_type, waterpoint_type_group

In [353]:
# unique waterpoint_type
features['waterpoint_type'].nunique(), features['waterpoint_type'].unique()

(7,
 array(['communal standpipe', 'communal standpipe multiple', 'hand pump',
        'other', 'improved spring', 'cattle trough', 'dam'], dtype=object))

In [354]:
features['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [355]:
# unique waterpoint_type_group
features['waterpoint_type_group'].nunique(), features['waterpoint_type_group'].unique()

(6,
 array(['communal standpipe', 'hand pump', 'other', 'improved spring',
        'cattle trough', 'dam'], dtype=object))

In [356]:
features['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

- Dropping waterpoint_type_group. Waterpoint_type is more specific. It differentiates the two communal standpipe cats (extra information).

In [357]:
# drop waterpoint_type_group
drops.append('waterpoint_type_group')

# Drop columns:

In [358]:
# cols to drop
drops, len(drops)

(['date_recorded',
  'construction_year',
  'wpt_name',
  'subvillage',
  'region',
  'ward',
  'recorded_by',
  'scheme_management',
  'management_group',
  'extraction_type_group',
  'payment_type',
  'quality_group',
  'quantity_group',
  'source_type',
  'waterpoint_type_group'],
 15)

- 15 columns removed.

# Export final df:

In [359]:
df = features.drop(drops, axis=1)
df.shape

(59400, 27)

- 25 features + id and status_group columns.

In [360]:
df.to_csv('data/features_cleaned.csv')